# **Context**

In [ ]:
!pip -q install translate

In [152]:
import json
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup
import re
from googlesearch import search

from translate import Translator

## **Downloading the data (json files)**

In [153]:
data_path = '/content/drive/MyDrive/Machine_Learning/Projects/vehicle_market_analysis/raw_data'

data = []

file_names = os.listdir(data_path)
for file_name in tqdm(file_names, desc='Files read'):
    with open(os.path.join(data_path, file_name)) as f:
        json_file = json.load(f)
        data += json_file

Files read: 100%|██████████| 740/740 [00:40<00:00, 18.19it/s] 


# **Data cleaning**

In [154]:
# Making Pandas DataFrame out of the data
vehicle_df = pd.DataFrame.from_dict(data)

In [155]:
# Removing duplicate values
vehicle_df = vehicle_df.drop_duplicates()

In [156]:
print('DataFrame shape:', vehicle_df.shape)

DataFrame shape: (13868, 13)


In [157]:
vehicle_df.head()

,title,price,telephone_number,Місто,Рік випуску,Пробіг,Модифікація,Двигун,Коробка,Привід,Колір,VIN,Кузов
0,Volkswagen Golf GTI 2.0 TSI 7-DSG (300 л.с.) 2022,Unknown price,380672349490,Киев,2022,29 тис. км.,2.0 TSI 7-DSG (300 л.с.),Бензиновий / 2.0 куб. см,Робот,Передній,NaN,NaN,NaN
1,Skoda Octavia 2017,530103,380678032322,Черкассы,2017,150 тис. км.,NaN,Бензиновий /,NaN,NaN,NaN,NaN,NaN
2,Volkswagen Jetta 2015,400483,380669202706,Запорожье,2015,184 тис. км.,NaN,NaN,NaN,NaN,синій,3VW267AJ8GM222832,NaN
3,Chery Amulet 2007,98167,380936786571,Хмельник,2007,40 тис. км.,NaN,Бензиновий /,Механіка,Передній,блакитний,NaN,NaN
4,Ford Escort 1991,10000,"380939148634, 380632935428",Черкассы,1991,300 тис. км.,NaN,Бензиновий / 1.4 л,Механіка,Передній,синій,NaN,Хетчбек


In [158]:
print('# of missing values in each column:')
vehicle_df.isna().sum()

# of missing values in each column:


title                   0
price                   0
telephone_number     4697
Місто                   0
Рік випуску             1
Пробіг                295
Модифікація         10964
Двигун               8066
Коробка              8111
Привід               8124
Колір               10079
VIN                  5883
Кузов               10057
dtype: int64

In [159]:
# Renaming the columns
new_column_names = {
    'Місто': 'car_location',
    'Рік випуску': 'manufacture_year',
    'Пробіг': 'mileage_thousand_km',
    'Кузов': 'car_body_type',
    'Колір': 'color',
    'Двигун': 'engine',
    'Коробка': 'transmission',
    'Привід': 'drive_unit',
    'Модифікація': 'modifications',
    'price': 'price_uah'
}

vehicle_df = vehicle_df.rename(columns=new_column_names)

In [160]:
print('New column names:', list(vehicle_df.columns))

New column names: ['title', 'price_uah', 'telephone_number', 'car_location', 'manufacture_year', 'mileage_thousand_km', 'modifications', 'engine', 'transmission', 'drive_unit', 'color', 'VIN', 'car_body_type']


**Cleaning 'car_location' column**

There are many unique entries in the data (shown in the cell below). These are the cars located in small towns and villages. These entries don't provide valuable information on their own. Because it's impossible to access how car location affects the price for these small towns.

However, to make car location info valuable, we will map each city and town to the corresponding region. In this way, we would be able to gain valuable insights.

In [161]:
vehicle_df.car_location.value_counts()

Киев          5969
Львов         2173
Одесса        1356
Винница        680
Харьков        528
              ... 
Бершадь          1
Снигирёвка       1
Макаров          1
Нежин            1
Копычинцы        1
Name: car_location, Length: 113, dtype: int64

For our purposes, we will define a function that makes a call to Wikipedia and retrieves the information about each location. After that we map each location to the corresponding region, that was mentioned on the location's Wikipedia page.

In [162]:
def get_car_location_region(car_location):
    query = f'Википедия населённый пункт {car_location} Украина'
    url = next(search(query, tld='com', num=1, stop=1))
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'lxml')

    try:
        result = soup.find('a', title=re.compile(r'Область')).parent.find_next_sibling().a.get('title')
    except AttributeError:
        result = car_location
    return result

In [163]:
# Getting locations names
car_locations = list(vehicle_df.car_location.value_counts().keys())

# Making a mapping of location to its region
car_location_regions = {}
for car_location in tqdm(car_locations, desc='Locations processed'):
    car_location_regions[car_location] = get_car_location_region(car_location)

Locations processed: 100%|██████████| 113/113 [05:15<00:00,  2.79s/it]


In [164]:
# Map locations to the regions
vehicle_df.car_location = vehicle_df.car_location.map(lambda x: car_location_regions[x])

In [166]:
vehicle_df.car_location.value_counts()

Киев                         5969
Львовская область            2252
Одесская область             1365
Винницкая область             694
Харьковская область           528
Днепропетровская область      405
Ивано-Франковская область     401
Волынская область             379
Черновицкая область           357
Киевская область              329
Запорожская область           285
Житомирская область           239
Закарпатская область          236
Кировоградская область        122
Черкасская область             97
Николаевская область           81
Полтавская область             28
Тернопольская область          25
Черниговская область           22
Ровненская область             18
Хмельницкая область            15
Сумская область                11
Донецкая область                5
Херсонская область              3
Новоград-Волынский              2
Name: car_location, dtype: int64

As we can see, some locations were not mapped. It is due to the recent renaming of the towns. Let's define its location by hand.

In [167]:
vehicle_df.loc[vehicle_df.car_location == 'Новоград-Волынский', 'car_location'] = 'Житомирская область'

Now, let's translate names of the regions to English.

In [168]:
# Making a translation mapping of region names to English
ru2eng_location_mapping = {}

translator= Translator(from_lang='ru', to_lang="en")
region_names = list(vehicle_df.car_location.unique())
for region_name in region_names:
    translation = translator.translate(region_name).split()
    if len(translation) > 1:
        translation[1] = 'Oblast'
    translation = ' '.join(translation)
    ru2eng_location_mapping[region_name] = translation

ru2eng_location_mapping['Киев'] = 'Kyiv'

In [169]:
# Translate region names
vehicle_df.car_location = vehicle_df.car_location.map(lambda x: ru2eng_location_mapping[x])

In [170]:
vehicle_df.car_location.value_counts()

Kyiv                      5969
Lviv Oblast               2252
Odessa Oblast             1365
Vinnytsia Oblast           694
Kharkov Oblast             528
Dnipropetrovsk Oblast      405
Ivano-Frankivsk Oblast     401
Volyn Oblast               379
Chernivtsi Oblast          357
Kiev Oblast                329
Zaporizhzhia Oblast        285
Zhytomyr Oblast            241
Zakarpattia Oblast         236
Kirovohrad Oblast          122
Cherkasy Oblast             97
Mykolaiv Oblast             81
Poltava Oblast              28
Ternopol Oblast             25
Chernihiv Oblast            22
Rivne Oblast                18
Khmelnytskyi Oblast         15
Sumy Oblast                 11
Donetsk Oblast               5
Kherson Oblast               3
Name: car_location, dtype: int64

**Cleaning 'mileage' column**

Clean columns from non-digit characters.

In [171]:
mileage_thousand_km = vehicle_df.mileage_thousand_km.str.split(expand=True).iloc[:, 0].str.strip()

Check for non-digit values.

In [172]:
mileage_thousand_km[~mileage_thousand_km.str.isdigit().astype(bool)].value_counts()

новий    1116
Name: 0, dtype: int64

There is an entry 'новий' which means new. Let's replace it with the value 0.

In [173]:
mileage_thousand_km[mileage_thousand_km == 'новий'] = 0

In [174]:
vehicle_df['mileage_thousand_km'] = mileage_thousand_km

**Cleaning 'car_body' column**

Let's check which car body types present in the data.

In [175]:
vehicle_df.car_body_type.value_counts()

Позашляховик              1111
Седан                      898
Кросовер                   572
Універсал                  487
Хетчбек                    346
Купе                        79
Ліфтбек                     75
Мінівен                     73
Фургон                      29
Кабриолет, Хетчбэк          23
Кабріолет                   23
Кабриолет, Купе             20
Пікап                       18
Мікроавтобус                18
Шасі                        15
Родстер                     10
Кабриолет, Внедорожник       7
Кабриолет, Седан             3
Кабриолет, Кабриолет         2
Тарга                        1
Борт                         1
Name: car_body_type, dtype: int64

Where there is two body types, let's pick first.

In [176]:
vehicle_df.car_body_type = vehicle_df.car_body_type.str.split(', ', expand=True).iloc[:, 0].str.strip()

In [177]:
# Define mapping by hand, because translator fails this task
ru2eng_body_type_mapping = {'Седан': 'Sedan',
                            'Універсал': 'Station wagon',
                            'Позашляховик': 'Off-road vehicle',
                            'Мінівен': 'Minivan',
                            'Кросовер': 'Crossover',
                            'Хетчбек': 'Hatchback',
                            'Ліфтбек': 'Liftback',
                            'Мікроавтобус': 'Minibus',
                            'Кабриолет': 'Convertible',
                            'Родстер': 'Roadster',
                            'Кабріолет': 'Convertible',
                            'Купе': 'Coupe',
                            'Шасі': 'Rolling chassis',
                            'Пікап': 'Pickup truck',
                            'Фургон': 'Van',
                            'Тарга': 'Targa top',
                            'Борт': 'Skirt'}

In [178]:
# Translate the car body types
vehicle_df.loc[~vehicle_df.car_body_type.isna(), 'car_body_type'] = vehicle_df.car_body_type[~vehicle_df.car_body_type.isna()].map(lambda x: ru2eng_body_type_mapping[x])

In [180]:
vehicle_df.car_body_type.value_counts()

Off-road vehicle    1111
Sedan                898
Crossover            572
Station wagon        487
Hatchback            346
Coupe                 79
Convertible           78
Liftback              75
Minivan               73
Van                   29
Minibus               18
Pickup truck          18
Rolling chassis       15
Roadster              10
Targa top              1
Skirt                  1
Name: car_body_type, dtype: int64

**Cleaning 'color' column**

In [181]:
vehicle_df.color.value_counts()

сірий           1079
чорний           982
білий            726
синій            370
червоний         206
сріблястий       101
коричневий        94
зелений           79
бежевий           75
вишневий          19
помаранчевий      18
жовтий            10
інший              9
золотистий         9
блакитний          7
фіолетовий         4
бронзовий          1
Name: color, dtype: int64

In [182]:
# Making a translation mapping of color names to English
ukr2eng_color_mapping = {}

translator= Translator(from_lang='ukr', to_lang="en")
for color in list(vehicle_df.color.unique()):
    try:
        translation = translator.translate(color)
        ukr2eng_color_mapping[color] = translation
    except Exception:
        continue

ukr2eng_color_mapping['золотистий'] = 'golden'
ukr2eng_color_mapping['бронзовий'] = 'bronze'

In [183]:
# Translate color names to English
vehicle_df.loc[~vehicle_df.color.isna(), 'color'] = vehicle_df.color[~vehicle_df.color.isna()].map(lambda x: ukr2eng_color_mapping[x])

In [184]:
vehicle_df.color = vehicle_df.color.str.strip()

In [185]:
vehicle_df.color.value_counts()

gray          1079
black          982
white          726
blue           370
red            206
Silver         101
brown           94
green           79
Beige           75
Plum            19
orange          18
yellow          10
other            9
golden           9
light blue       7
purple           4
bronze           1
Name: color, dtype: int64

**Cleaning 'transmission' column**

In [186]:
vehicle_df.transmission.value_counts()

Автомат     4000
Механіка     994
Робот        359
Механика     151
Вариатор     148
Варіатор     105
Name: transmission, dtype: int64

In [187]:
ukr2eng_transmission_mapping = {
    'Автомат': 'automatic',
    'Механіка': 'mechanical',
    'Робот': 'robot',
    'Вариатор': 'automatic',
    'Механика': 'mechanical',
    'Варіатор': 'automatic'
    }

In [188]:
vehicle_df.loc[~vehicle_df.transmission.isna(), 'transmission'] = vehicle_df.transmission[~vehicle_df.transmission.isna()].map(lambda x: ukr2eng_transmission_mapping[x])

In [189]:
vehicle_df.transmission.value_counts()

automatic     4253
mechanical    1145
robot          359
Name: transmission, dtype: int64

**Cleaning 'drive_unit' column**

In [190]:
vehicle_df.drive_unit.value_counts()

Повний                     2999
Передній                   2280
Задній                      391
Повний що підключається      74
Name: drive_unit, dtype: int64

In [191]:
ukr2eng_drive_unit_mapping = {
    'Повний': 'All-wheel drive',
    'Передній': 'Front-wheel drive',
    'Задній': 'Rear-wheel drive',
    'Повний що підключається': 'All-wheel drive'
}

In [192]:
vehicle_df.loc[~vehicle_df.drive_unit.isna(), 'drive_unit'] = vehicle_df.drive_unit[~vehicle_df.drive_unit.isna()].map(lambda x: ukr2eng_drive_unit_mapping[x])

In [193]:
vehicle_df.drive_unit.value_counts()

All-wheel drive      3073
Front-wheel drive    2280
Rear-wheel drive      391
Name: drive_unit, dtype: int64

**Cleaning 'engine' column**

In [194]:
vehicle_df.engine.value_counts()

Дизельний / 3.0 куб. см          336
Електричний /                    291
Бензиновий /                     182
Дизельний / 2.0 куб. см          171
Дизельный / 2.0 л                143
                                ... 
Дизельний / 3 л / 340 к.с.         1
Бензиновий / 3.2 л / 233 к.с.      1
Бензиновий / 2.8 л / 204 к.с.      1
Бензиновий / 1 л / 55 к.с.         1
Бензиновий / 4.4 л / 315 к.с.      1
Name: engine, Length: 1215, dtype: int64

Engine columns can be separated into three columns:
* engine type
* engine volume
* number of horse powers of engine

In [195]:
engine_type = vehicle_df.engine.str.split('/', expand=True).loc[:, 0].str.strip()
engine_volume = vehicle_df.engine.str.split('/', expand=True).loc[:, 1].str.strip()
engine_horse_powers = vehicle_df.engine.str.split('/', expand=True).loc[:, 2].str.strip().str.split(expand=True).loc[:, 0]

In [196]:
vehicle_df['engine_type'] = engine_type
vehicle_df['engine_volume'] = engine_volume
vehicle_df['engine_horse_powers'] = engine_horse_powers

**Cleaning 'engine_type' column**

In [197]:
vehicle_df.engine_type.value_counts()

Бензиновий       2137
Дизельний        1635
Бензиновый        462
Газ               411
Дизельный         375
Електричний       296
Гібридний         258
Гибридный         188
Электрический      40
Name: engine_type, dtype: int64

In [198]:
ukr2eng_engine_type_mapping = {
    'Бензиновий': 'petrol',
    'Дизельний': 'diesel',
    'Бензиновый': 'petrol',
    'Дизельный': 'diesel',
    'Газ': 'gas',
    'Електричний': 'electric',
    'Гібридний': 'hybrid',
    'Гибридный': 'hybrid',
    'Электрический': 'electric'
    }

In [199]:
vehicle_df.loc[~vehicle_df.engine_type.isna(), 'engine_type'] = vehicle_df.engine_type[~vehicle_df.engine_type.isna()].map(lambda x: ukr2eng_engine_type_mapping[x])

In [200]:
vehicle_df.engine_type.value_counts()

petrol      2599
diesel      2010
hybrid       446
gas          411
electric     336
Name: engine_type, dtype: int64

**Cleaning 'engine_volume' column**

In [202]:
engine_volume.unique()

array(['2.0 куб. см', '', nan, '1.4 л', '1.6 л', '1.3 л', '3.0 куб. см',
       '2.4 л', '2 л', '4.4 куб. см', '3.8 лГБО', '1.4 лГБО', '3.4 лГБО',
       'ГБО', '3.0 л', '2.8 л', '2.0 л', '2.9 л', '3.5 л', '4.0 л', '3 л',
       '3.6 л', '2.4 лГБО', '2.5 л', '1 л', '5 л', '1600 лГБО', '1.8 л',
       '1.5 л', '2.2 л', '1.2 л', '3.2 л', '306 к.с.', '1600 л', '2.1 л',
       '4.6 л', '4.0 куб. см', '2.3 л', '20 л', '2.7 л', '4.8 л', '3.7 л',
       '4.5 л', '1.1 л', '4.7 л', '4.4 л', '3.3 л', '1.9 л',
       '2.5 куб. см', '0.6 л', '6 л', '1845 л', '5.5 л', '5.7 л', '0.8 л',
       '0.7 л', '3 куб. см', '4.2 л', '0.9 л', '4 л', '1.0 л',
       '2.2 куб. см', '1500 л', '2000 л', '3,0 куб. см', '5.2 куб. см',
       '2.9 куб. см', '5.5 куб. см', '6.0 куб. см', '4395 куб. см',
       '5.2 л', '6.0 л', '1.5 куб. см', '6.6 куб. см', '4 куб. см',
       '1.4 куб. см', '5.0 куб. см', '6.6 л', '1.3 куб. см',
       '1991 куб. см', '4,4 куб. см', '2 куб. см', '1950 куб. см',
       '5.0 л', '1.8 

To make an informative column, we should extract number (engine_volume_value) and a measure units for this value (engine_volume_coef).

Making 'engine_volume_value' series

In [203]:
engine_volume_value = vehicle_df.engine_volume.str.split(expand=True, n=1).loc[:, 0]
engine_volume_value = engine_volume_value.str.replace(',', '.')

pattern = re.compile(r'^(\d)+(\.)?(\d)*')
engine_volume_value[~engine_volume_value.isna()] = engine_volume_value[~engine_volume_value.isna()].apply(lambda x: np.nan if not bool(re.match(pattern, x)) else float(x))

Making 'engine_volume_coef' series

In [204]:
vehicle_df.engine_volume.str.split(expand=True, n=1).loc[:, 1].value_counts()

л          3842
куб. см    1169
лГБО         44
к.с.          2
Name: 1, dtype: int64

In [205]:
engine_volume_mapping = {
    'л': 1,
    'куб. см': 0.001,
    'лГБО': 1,
    'к.с.': np.nan
}

In [206]:
engine_volume_coef = vehicle_df.engine_volume.str.split(expand=True, n=1).loc[:, 1].str.strip()
engine_volume_coef[~engine_volume_coef.isna()] = engine_volume_coef[~engine_volume_coef.isna()].map(lambda x: engine_volume_mapping[x])

Transforming 'engine_volume' column.

In [207]:
engine_volume = engine_volume_coef * engine_volume_value
vehicle_df['engine_volume'] = engine_volume

In [208]:
# Dropping redundant column
vehicle_df = vehicle_df.drop(columns=['engine'], axis=1)

**Cleaning 'price_uah' column**

For convenience, let's move a target column to the last position.

In [209]:
price_col = vehicle_df.pop('price_uah')
vehicle_df.insert(vehicle_df.shape[1], 'price_uah', price_col)

In [210]:
vehicle_df.head()

,title,telephone_number,car_location,manufacture_year,mileage_thousand_km,modifications,transmission,drive_unit,color,VIN,car_body_type,engine_type,engine_volume,engine_horse_powers,price_uah
0,Volkswagen Golf GTI 2.0 TSI 7-DSG (300 л.с.) 2022,380672349490,Kyiv,2022,29,2.0 TSI 7-DSG (300 л.с.),robot,Front-wheel drive,NaN,NaN,NaN,petrol,0.002,None,Unknown price
1,Skoda Octavia 2017,380678032322,Cherkasy Oblast,2017,150,NaN,NaN,NaN,NaN,NaN,NaN,petrol,NaN,None,530103
2,Volkswagen Jetta 2015,380669202706,Zaporizhzhia Oblast,2015,184,NaN,NaN,NaN,blue,3VW267AJ8GM222832,NaN,NaN,NaN,NaN,400483
3,Chery Amulet 2007,380936786571,Vinnytsia Oblast,2007,40,NaN,mechanical,Front-wheel drive,light blue,NaN,NaN,petrol,NaN,None,98167
4,Ford Escort 1991,"380939148634, 380632935428",Cherkasy Oblast,1991,300,NaN,mechanical,Front-wheel drive,blue,NaN,Hatchback,petrol,1.4,None,10000


Dataset is clean and ready for ML stage of a production.

**Saving cleaned dataset**

In [212]:
vehicle_df.to_csv('/content/drive/MyDrive/Machine_Learning/Projects/vehicle_market_analysis/vehicle_data/vehicle_data.csv')